In [25]:
import numpy as np
import cvxpy as cp

In [30]:
# Definir variáveis k1, k2, m1, m2, c0
k1, k2, m1, m2, c0 = 1, 1, 1, 0.5, 2
# Definir as matrizes A, B e C
A = np.array([[0, 0, 1, 0],
              [0, 0, 0, 1],
              [-(k1+k2)/m1, k2/m1, -c0/m1, 0],
              [k2/m2, -k2/m2, 0, -c0/m2]])

B = np.array([[0], [0], [1/m1], [0]])
C = np.array([[0, 1, 0, 0]])

n, p = A.shape[0], B.shape[1]
# Definir a variável P.
P = cp.Variable((n, n), symmetric=True)
D = cp.Variable((p, p), symmetric=True)

mu = cp.Variable()
# Definir a função objetivo: min (mu)
objective = cp.Minimize(mu)

# --- CORREÇÃO AQUI ---
# Definir os blocos da LMI
M11 = A.T @ P + P @ A
M12 = P @ B
M13 = C.T
M21 = B.T @ P
M22 = - mu * np.eye(p)
M23 =  D.T
M31 = C
M32 = D
M33 = np.eye(p)

# Construir a matriz de blocos LMI
LMI = cp.bmat([
    [M11, M12, M13],
    [M21, M22, M23],
    [M31, M32, M33]
])

# Definir as restrições corrigidas
constraints = [P >> 0,
               LMI << 0]

# Criar e resolver o problema de otimização
problem = cp.Problem(objective, constraints)
problem.solve()

# Verificar o status da solução
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
    print("Status:", problem.status, "\n")
    # O objetivo é minimizar 'mu', não Tr(P)
    print("Valor ótimo de mu:")
    print(problem.value)
    print("\nMatriz P:")
    print(P.value)
    print("\nMatriz D:")
    print(D.value)
else:
    print("O problema não tem uma solução ótima. Status:", problem.status)



Status: optimal_inaccurate 

Valor ótimo de mu:
226958.349620318

Matriz P:
[[ 6117409.47498679 -5594816.28041807  -806608.58980425 -9807510.03694721]
 [-5594816.28041807  5555171.53963885  1156280.25793024  9867240.97217433]
 [ -806608.58980425  1156280.25793024   782889.37881289  2423532.82120573]
 [-9807510.03694721  9867240.97217433  2423532.82120573 19703794.81892901]]

Matriz D:
[[-260.54097608]]


c:\CONTROLE_ROBUSTO\venv\Lib\site-packages\cvxpy\problems\problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
